# Предобработка постов

In [1]:
# импорт библиотек
import ast
import numpy as np
import pandas as pd

##  Предобработка постов, собранных своей командой (на примере файла с информацией о разработчиках)

In [2]:
# указание пути к данным для предобработки
DATA_PATH = 'developers.csv'

В части собранных постов комментарии представляют собой текст формата "3 комментария". Создадим функцию `get_comment_cnt` для сохранения количества комментариев в формате числа:

In [3]:
# создание функции get_comment_cnt
def get_comment_cnt(comment_cnt) -> int:
    '''
    Функция вычленяет количество комментариев и возвращает его в формате int.
    
    Аргументы:
    - comment_cnt - значение количества комментариев.
    '''
    
    if type(comment_cnt) == str:
        toy = comment_cnt.split()
        return int(comment_cnt[0])
    else:
        return comment_cnt

In [4]:
data = pd.read_csv(DATA_PATH, index_col = 0) # загрузка данных
data = data[data.works_at.str.contains('dev', case = False)] # уберем нерелевантные профили (профили по другим профессиям)
data.head() # вывод на экран первых пяти строк датасета

,profile_url,name,works_at,exp_list,posts,reactions,comments,post_url
0,https://www.linkedin.com/in/ruslandubrovin/,Руслан Дубровин,Software Developer – Yandex,"['Software Developer', 'Yandex', 'март 2019 г....",[],[],[],[]
1,https://www.linkedin.com/in/grigory-kostin-aaa...,Grigory Kostin,Developer at Yandex,"['Developer', 'Yandex', 'янв. 2015 г. – настоя...",[],[],[],[]
2,https://www.linkedin.com/in/bard-10x/,Денис Ковалев,iOS App Developer – Tinkoff,"['iOS Developer', 'Tinkoff · Полный рабочий д...",[],[],[],[]
3,https://www.linkedin.com/in/kirill-khrustitski...,Kirill Khrustitskii,Developer,"['Разработчик', 'МТС · Полный рабочий день', '...","['Здравствуйте, я в поиске новой карьерной воз...","['1', None, None, None, '1', None, None, None,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",['https://www.linkedin.com/feed/update/urn:li:...
4,https://www.linkedin.com/in/artem-lesnoy/,Artem Lesnoy,Senior Frontend Developer – Yandex,"['Frontend Developer', 'Yandex · Полный рабочи...",[],[],[],[]


Для Topic modeling'a мы хотим, чтобы один объект датасета содержал информацию об одном посте. Реализуем это:

In [5]:
exp_col = ['posts', 'reactions', 'comments', 'post_url'] # создание списка с названиями колонок, которые нужно "развернуть" в отдельные строки
data[exp_col] = data[exp_col].applymap(ast.literal_eval) # обновление информации о типе данных
data['posts_cnt'] = data['posts'].apply(len) # сохранение количества постов по каждому пользователю в отдельную переменную
data_exp = data.apply(pd.Series.explode) # разворачивание постов в отдельные строки
data_exp.head() # вывод на экран первых пяти строк "развернутого" датасета

,profile_url,name,works_at,exp_list,posts,reactions,comments,post_url,posts_cnt
0,https://www.linkedin.com/in/ruslandubrovin/,Руслан Дубровин,Software Developer – Yandex,"['Software Developer', 'Yandex', 'март 2019 г....",NaN,NaN,NaN,NaN,0
1,https://www.linkedin.com/in/grigory-kostin-aaa...,Grigory Kostin,Developer at Yandex,"['Developer', 'Yandex', 'янв. 2015 г. – настоя...",NaN,NaN,NaN,NaN,0
2,https://www.linkedin.com/in/bard-10x/,Денис Ковалев,iOS App Developer – Tinkoff,"['iOS Developer', 'Tinkoff · Полный рабочий д...",NaN,NaN,NaN,NaN,0
3,https://www.linkedin.com/in/kirill-khrustitski...,Kirill Khrustitskii,Developer,"['Разработчик', 'МТС · Полный рабочий день', '...","Здравствуйте, я в поиске новой карьерной возмо...",1,0,https://www.linkedin.com/feed/update/urn:li:ac...,11
3,https://www.linkedin.com/in/kirill-khrustitski...,Kirill Khrustitskii,Developer,"['Разработчик', 'МТС · Полный рабочий день', '...",Unity Game Development Mini-Degree,None,0,https://www.linkedin.com/feed/update/urn:li:ac...,11


В результате разворачивания постов в отдельные строки образовались пропуски в значениях столбцов `posts`, `post_url`, `reactions` и `comments` для случаев, когда у пользователя не было постов. Заполним их:

In [6]:
# заполнение пропусков
data_exp.fillna(value = {'posts'    : 'нет постов',
                         'post_url' : 'нет постов',
                         'reactions': 0,
                         'comments' : 0
                        },
                inplace = True)

# проверка заполения пропусков
data_exp.isna().sum()

profile_url    0
name           0
works_at       0
exp_list       0
posts          0
reactions      0
comments       0
post_url       0
posts_cnt      0
dtype: int64

In [7]:
# удаление профилей без постов
data_exp = data_exp[data_exp.posts != 'нет постов']

In [8]:
# применение функции 
data_exp['comments'] = data_exp['comments'].apply(get_comment_cnt)

In [9]:
# переименование столбцов
data_exp.rename(columns = {
    'posts'    : 'post',
    'reactions': 'reactions_cnt',
    'comments' : 'comments_cnt'},
                inplace = True)

В отличие от других команд мы не собирали количество репостов, но для целей объединения датасетов создадим соответствующий столбец и заполним его пропусками:

In [10]:
# создание столбца reposts_cnt
data_exp['reposts_cnt'] = np.nan

Оставим в датасете только информацию, связанную с постами, и url профилей:

In [11]:
# создадние списка с названиями столбцов, которые необходимо оставить
post_cols = ['profile_url', 'post_url', 'post', 'reactions_cnt', 'comments_cnt', 'reposts_cnt']

# сокращение количества столбцов
data_exp = data_exp[post_cols]

# проверка сокращения количества столбцов
print(data_exp.columns)

Index(['profile_url', 'post_url', 'post', 'reactions_cnt', 'comments_cnt',
       'reposts_cnt'],
      dtype='object')


In [12]:
# сохранение обработанного датасета
data_exp.to_csv('developers_prep.csv', sep=';')

In [13]:
# загрузка всех предобработанных файлов команды
developers = pd.read_csv('developers_prep.csv', index_col=0, sep=';')
ios_developers = pd.read_csv('ios_developers_prep.csv', index_col=0, sep=';')

In [14]:
# объединение датасетов, собранных разными членами команды
team_data = pd.concat([developers, ios_developers])

team_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 3 to 145
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   profile_url    117 non-null    object 
 1   post_url       117 non-null    object 
 2   post           117 non-null    object 
 3   reactions_cnt  117 non-null    int64  
 4   comments_cnt   117 non-null    int64  
 5   reposts_cnt    0 non-null      float64
dtypes: float64(1), int64(2), object(3)
memory usage: 6.4+ KB


In [15]:
# сохранение объединенного датасета
team_data.to_csv('team_8_posts.csv', sep=';')

#  Предобработка постов, собранных другими командами (на примере постов команды 10)

In [16]:
# указание пути к данным
PATH = 'Команда №10.csv'

In [17]:
data = pd.read_csv(PATH, sep=';') # загрузка данных
data.reset_index(drop=True, inplace = True) # сброс индексов
data.head() # вывод на экран первых пяти строк

,account_link,search_keywords,name,title,works_at,intro,experience,place,posts_cnt,post_text,reaction_cnt,comments_cnt,repost_cnt
0,https://www.linkedin.com/in/nikita-balykin-b01...,frontend,Nikita Balykin,Frontend Web Developer,ФКУ Соцтех,Development of internal company products.stack...,3.0,"Бали, Индонезия",2,"Здравствуйте, я в поиске новой карьерной возмо...",2.0,NaN,1.0
1,https://www.linkedin.com/in/anton-zaglumonin,lead web developer,Anton Zaglumonin,Lead Frontend Developer – Web Enthusiast,"ПАО ""Ростелеком""​",Привет!Я инженер-разработчик с более чем 10-ле...,12.0,"Республика Мордовия, Россия",22,Vitali Kashko #Отзывы с благодарностью Вы вдох...,NaN,NaN,NaN
2,https://www.linkedin.com/in/anastasiya-saydazi...,frontend,Анастасия Сайдазимова,"Frontend Developer | React, Redux, JavaScript,...",NaN,3 года назад начала изучать Frontend-разработк...,1.0,"Санкт-Петербург, Санкт-Петербург, Россия",158,UPD: Всем огромное спасибо! Работа найдена 😎 П...,1.0,8.0,4.0
3,https://www.linkedin.com/in/ankhanhi,ML engineer,Анна Хан,Data scientist | Analyst | ML-engineer,Московский Государственный Институт Электронно...,👋 Привет! Я студентка 3 курса программной инже...,2.0,"Москва, Москва, Россия",9,"Никогда не догадалась, что задача о многоруких...",2.0,NaN,NaN
4,https://www.linkedin.com/in/oleh-yudin-a8643bb2,senior ML engineer,Oleh Yudin,Senior ML Research Engineer,Moscow Institute of Physics and Technology (St...,Knowledge of one principle frees thousands of ...,4.0,Агломерация Днепра,6,Real time bidding. New loss function.Online ad...,1.0,NaN,NaN


Команда 10 не собирала ссылки постов, поэтому для объединения датасетом создадим соответствующий столбец и заполним пропусками:

In [18]:
# создание столбца post_url
data['post_url'] = np.nan

In [19]:
# переименование столбцов
data.rename(columns = {
    'account_link' : 'profile_url',
    'post_text' : 'post',
    'reaction_cnt' : 'reactions_cnt',
    'repost_cnt' : 'reposts_cnt'
    
},
            inplace = True)

# проверка переименования столбцов
data.columns

Index(['profile_url', 'search_keywords', 'name', 'title', 'works_at', 'intro',
       'experience', 'place', 'posts_cnt', 'post', 'reactions_cnt',
       'comments_cnt', 'reposts_cnt', 'post_url'],
      dtype='object')

In [20]:
# создание списка с названиями столбцов, которые нужно оставить
cols = ['profile_url', 'post_url', 'post', 'reactions_cnt', 'comments_cnt', 'reposts_cnt']

# сокращение количества столбцов
data = data[cols]

# проверка сокращения количества столбцов
data.columns

Index(['profile_url', 'post_url', 'post', 'reactions_cnt', 'comments_cnt',
       'reposts_cnt'],
      dtype='object')

In [21]:
# сохранение предобработанного датасета
data.to_csv('team_10_posts.csv', sep=';')

In [22]:
# загрузка предобработанных датасетов всех команд, которые участовали в обмене анкетами
data_2  = pd.read_csv('team_2_posts.csv', sep=';', index_col = 0)
data_3  = pd.read_csv('team_3_posts.csv', sep=';', index_col = 0)
data_8  = pd.read_csv('team_8_posts.csv', sep=';', index_col = 0)
data_10 = pd.read_csv('team_10_posts.csv', sep=';', index_col = 0)

In [23]:
# объединение датасетов
data_to_con = [data_2, data_3, data_8, data_10]
data_total = pd.concat(data_to_con)
data_total.head()

,profile_url,post_url,post,reactions_cnt,comments_cnt,reposts_cnt
0,https://www.linkedin.com/in/ali-wodan/recent-a...,NaN,Кстати говоря. Теперь подкаст Миражи доступен ...,1,0.0,0.0
1,https://www.linkedin.com/in/ali-wodan/recent-a...,NaN,I’m #hiring. Know anyone who might be interested?,1,0.0,0.0
2,https://www.linkedin.com/in/ali-wodan/recent-a...,NaN,Хэй честной народ! Ищу векторного иллюстратора...,6,0.0,0.0
3,https://www.linkedin.com/in/ali-wodan/recent-a...,NaN,"Новый пост подкаста ""Миражи"" на всех платформа...",1,0.0,0.0
4,https://www.linkedin.com/in/ali-wodan/recent-a...,NaN,"Подкаст Миражи, Эпизод 13Ошибка невозвратных з...",2,0.0,0.0


In [24]:
# проверка на дубликаты
data_total.duplicated().sum()

123

In [25]:
# вывод на экран первых 10 дубликатов
data_total[data_total.duplicated()].sort_values('post').head(10)

,profile_url,post_url,post,reactions_cnt,comments_cnt,reposts_cnt
232,https://www.linkedin.com/in/alexander-lobtsov,NaN,#OpenToWork Identifying potential Developer Ad...,27,0.0,10.0
221,https://www.linkedin.com/in/alexander-lobtsov,NaN,#OpenToWork Identifying potential Developer Ad...,27,0.0,10.0
233,https://www.linkedin.com/in/alexander-lobtsov,NaN,#OpenToWork Identifying potential Developer Ad...,27,0.0,10.0
412,https://www.linkedin.com/in/ildar-abdullin/rec...,NaN,#hr #glossary,0,0.0,0.0
413,https://www.linkedin.com/in/ildar-abdullin/rec...,NaN,#hr #glossary,0,0.0,0.0
418,https://www.linkedin.com/in/ildar-abdullin/rec...,NaN,#hr #glossary,0,0.0,0.0
409,https://www.linkedin.com/in/ildar-abdullin/rec...,NaN,#lean #glossary,1,0.0,0.0
586,https://www.linkedin.com/in/vladimir-yelizarof...,NaN,#mvp,0,0.0,0.0
582,https://www.linkedin.com/in/vladimir-yelizarof...,NaN,#автоматизация,1,0.0,0.0
584,https://www.linkedin.com/in/vladimir-yelizarof...,NaN,#автоматизация,2,0.0,0.0


In [26]:
# удаление дубликатов
data_total.drop_duplicates(inplace = True, ignore_index = True)

In [27]:
# сохранение объединенного датасета
data_total.to_csv('teams_2_3_8_10_posts.csv', sep=';')